In [2]:
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.nn import GCNConv

# Define the GCN layer and the network model
class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        
        # Define two GCN layers
        self.conv1 = GCNConv(in_channels, 16)  # 16 is the number of hidden features
        self.conv2 = GCNConv(16, out_channels) # Output layer

    def forward(self, x, edge_index):
        # x is the node feature matrix of shape [num_nodes, num_features]
        # edge_index is the graph connectivity in COO format (2 x num_edges)

        # Apply first graph convolution layer + ReLU activation
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        
        # Apply second graph convolution layer
        x = self.conv2(x, edge_index)
        
        return x

# Example usage
num_nodes = 15
num_features = 10  # Example feature size (e.g., node features are 3-dimensional)
num_classes = 1   # Number of classes for classification task

# Create random node features and a random graph structure
x = torch.randn((num_nodes, num_features))  # Random features for 10 nodes
print("x", x.shape)
edge_index = torch.randint(0, num_nodes, (2, 20))  # Random edges for the graph (20 edges)

print("edge_index", edge_index, edge_index.shape)

# Create the GCN model
model = GCN(in_channels=num_features, out_channels=num_classes)

# Forward pass through the model
out = model(x, edge_index)

# Print the output (logits for node classification)
print(out)


x torch.Size([15, 10])
edge_index tensor([[ 2, 10, 10, 13,  4, 13,  5, 12, 10,  7,  6,  9,  7,  6,  7,  4, 10, 10,
          7, 11],
        [11,  0, 10,  2,  8,  6,  2, 10,  8,  8,  6, 12, 12, 11, 14, 10, 14, 11,
          8,  1]]) torch.Size([2, 20])
tensor([[-0.5094],
        [-0.3260],
        [-1.6335],
        [-0.0818],
        [-0.7926],
        [-1.2286],
        [-0.8745],
        [-0.0526],
        [-0.7923],
        [ 0.3853],
        [-0.7766],
        [-0.7685],
        [ 0.2455],
        [-0.9333],
        [-0.4082]], grad_fn=<AddBackward0>)


In [4]:
import torch.optim as optim

# Assume we have labels for the nodes for classification (e.g., binary classification)
labels = torch.randint(0, num_classes, (num_nodes,))  # Random labels for example
print(labels)

# Create an optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

tensor([1, 0, 1, 1, 0, 1, 0, 0, 0, 0])


In [22]:
model.train()
for i in range(10):
    # Training loop (just one step for simplicity)
    optimizer.zero_grad()
    out = model(x, edge_index)
    loss = torch.nn.functional.cross_entropy(out, labels)
    loss.backward()
    optimizer.step()

    print(f"Training loss: {loss.item()}")

Training loss: 0.23366816341876984
Training loss: 0.23132281005382538
Training loss: 0.2289263755083084
Training loss: 0.22651085257530212
Training loss: 0.22406795620918274
Training loss: 0.22155213356018066
Training loss: 0.21897420287132263
Training loss: 0.2167687863111496
Training loss: 0.21493549644947052
Training loss: 0.2131083458662033


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset

# Load a dataset of graphs (e.g., MUTAG)
dataset = TUDataset(root='/tmp/MUTAG', name='MUTAG')

# Split dataset
train_dataset = dataset[:150]
test_dataset = dataset[150:]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define GCN
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        print("x", x.shape, "edge_index", edge_index.shape)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        # Pool over nodes for each graph in batch
        x = global_mean_pool(x, batch)  

        x = self.lin(x)
        return x

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train loop
for epoch in range(1, 201):
    model.train()
    for data in train_loader:
        out = model(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')


/home/agaru/anaconda3/envs/cvf/lib/python3.13/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


x torch.Size([569, 7]) edge_index torch.Size([2, 1262])
x torch.Size([608, 7]) edge_index torch.Size([2, 1354])
x torch.Size([545, 7]) edge_index torch.Size([2, 1196])
x torch.Size([555, 7]) edge_index torch.Size([2, 1214])
x torch.Size([407, 7]) edge_index torch.Size([2, 904])
Epoch 001, Loss: 0.7072
x torch.Size([565, 7]) edge_index torch.Size([2, 1256])
x torch.Size([552, 7]) edge_index torch.Size([2, 1216])
x torch.Size([585, 7]) edge_index torch.Size([2, 1278])
x torch.Size([634, 7]) edge_index torch.Size([2, 1426])
x torch.Size([348, 7]) edge_index torch.Size([2, 754])
Epoch 002, Loss: 0.6483
x torch.Size([595, 7]) edge_index torch.Size([2, 1328])
x torch.Size([583, 7]) edge_index torch.Size([2, 1282])
x torch.Size([573, 7]) edge_index torch.Size([2, 1264])
x torch.Size([548, 7]) edge_index torch.Size([2, 1206])
x torch.Size([385, 7]) edge_index torch.Size([2, 850])
Epoch 003, Loss: 0.6096
x torch.Size([540, 7]) edge_index torch.Size([2, 1188])
x torch.Size([563, 7]) edge_index t